In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [49]:
data = pd.read_csv('T100_2019.csv')
data = data.dropna(subset=['UNIQUE_CARRIER_NAME', 'ORIGIN', 'DEST'])

In [50]:
data['LOAD_FACTOR'] = data['PASSENGERS']/data['SEATS']
data[data['LOAD_FACTOR']>1]=1

In [51]:
# list(data)

In [59]:
scheduledflights = data[(data['CLASS'].isin(['A','C','E','F'])) & (data['SEATS']>0) & (data['DEPARTURES_SCHEDULED']>=5)] #Select only scheduled commercial flights and flights with available seats
scheduledflights = scheduledflights[scheduledflights['LOAD_FACTOR']<=1] #Get rid of cases where there are more passengers than seats
scheduledflights = scheduledflights.reset_index(drop=True)
# scheduledflights = scheduledflights[scheduledflights['SEATS']>50]

In [69]:
# datasubset = scheduledflights[['UNIQUE_CARRIER_NAME','ORIGIN','DEST','AIRCRAFT_TYPE','DEPARTURES_SCHEDULED','DISTANCE','MONTH']]
datasubset = scheduledflights[['UNIQUE_CARRIER_NAME','ORIGIN','DEST','DEPARTURES_SCHEDULED','MONTH']]

In [70]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('OH-ENCODE', OneHotEncoder(), [0,1,2,3])], remainder = 'drop')
X = ct.fit_transform(datasubset)
y = scheduledflights['LOAD_FACTOR']*100
pickle.dump(ct, open('OneHotEncoder-2.pkl', 'wb'))

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

## SGD Regressor

In [9]:
# from sklearn.linear_model import SGDRegressor
# from sklearn.metrics import mean_squared_error

# SGD_model = SGDRegressor()
# SGD_model.fit(X_train, y_train)
# score = SGD_model.score(X_train, y_train)
# print("R-squared:", score)
# y_pred = SGD_model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print("MSE: ", mse)

## SGD Regressor GridSearch

In [10]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import SGDRegressor
# from sklearn.metrics import mean_squared_error

# parameters = {'penalty':['elasticnet'], 
#               'alpha':[0.000001, 0.000005, 0.00001, 0.00005, 0.0001], 
#               'l1_ratio':[0, 0.2, 0.4, 0.6, 0.8, 1], 
#               'max_iter':[1000,2500,5000,10000]}
# sgdr = SGDRegressor()
# best_sgdr = GridSearchCV(sgdr, parameters, verbose=4, n_jobs = 5, cv = 3)
# best_sgdr.fit(X_train, y_train)

# print(best_sgdr.best_params_)

# import csv
# with open('results_cv_SGDregressor.csv', 'w') as csvfile:
#     writer = csv.DictWriter(csvfile, fieldnames=best_sgdr.cv_results_.keys())
#     writer.writeheader()
#     writer.writerow(best_sgdr.cv_results_)

# y_pred = best_sgdr.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print("MSE: ", mse)

## Ridge Regressor

In [11]:
# from sklearn.linear_model import Ridge
# from sklearn.metrics import mean_squared_error

# Ridge_model = Ridge()
# Ridge_model.fit(X_train, y_train)
# score = Ridge_model.score(X_train, y_train)
# print("R-squared:", score)
# y_pred = Ridge_model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print("MSE: ", mse)

## Ridge Grid Search

In [12]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import Ridge
# from sklearn.metrics import mean_squared_error

# parameters = {'alpha':[0.1, 0.5, 1, 5, 10], 
#               'max_iter':[1000,2500,5000,10000],
#               'tol':[0.00005, 0.0005, 0.0001, 0.005]}
# rr = Ridge()
# best_rr = GridSearchCV(rr, parameters, verbose=4, n_jobs = 5, cv = 3)
# best_rr.fit(X_train, y_train)

# print(best_rr.best_params_)

# import csv
# with open('results_cv_Ridgeregressor.csv', 'w') as csvfile:
#     writer = csv.DictWriter(csvfile, fieldnames=best_rr.cv_results_.keys())
#     writer.writeheader()
#     writer.writerow(best_rr.cv_results_)

# y_pred = best_rr.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print("MSE: ", mse)

## Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pickle

# RF_model = RandomForestRegressor(max_depth=300, max_features='log2', n_estimators=200)
RF_model = RandomForestRegressor()
RF_model.fit(X_train, y_train)
score = RF_model.score(X_train, y_train)
print("R-squared:", score)
y_pred = RF_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)
pickle.dump(RF_model, open('Load_Factor_RFR_Model_2.pkl', 'wb'))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

parameters = {'n_estimators':[200, 300, 400, 500], 'max_depth':[None, 200, 300, 400, 500], 'max_features':[None, 'auto', 'log2']}
rfr = RandomForestRegressor()
cv = GridSearchCV(rfr, parameters, verbose=4, n_jobs = 5, cv = 3)
cv.fit(X_train, y_train)
# Fitting 3 folds for each of 75 candidates, totalling 225 fits
# [Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
# [Parallel(n_jobs=5)]: Done  22 tasks      | elapsed: 54.2min
# [Parallel(n_jobs=5)]: Done 118 tasks      | elapsed: 161.1min
# [Parallel(n_jobs=5)]: Done 225 out of 225 | elapsed: 382.4min finished


print(cv.best_params_)
#{'max_depth': 300, 'max_features': 'log2', 'n_estimators': 200}

import csv
with open('results_cv_RFregressor2019.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=cv.cv_results_.keys())
    writer.writeheader()
    writer.writerow(cv.cv_results_)

y_pred = cv.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)
# MSE:  144.03809115310145

Fitting 3 folds for each of 60 candidates, totalling 180 fits


In [61]:
import csv

carriers = np.sort(datasubset['UNIQUE_CARRIER_NAME'].unique())
with open('carrier_names.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['No Selection'])
    for item in carriers:
        writer.writerow([item])

origins = np.sort(datasubset['ORIGIN'].unique())
with open('origins.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for item in origins:
        writer.writerow([item])

destinations = np.sort(datasubset['DEST'].unique())
with open('destinations.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for item in destinations:
        writer.writerow([item])

datasubset.to_csv('All_Flights.csv', index = False)

In [ ]:
# def getflightdata(data, origin, destination, carrier, month):
#     if carrier == 'No Selection':
#         subdata = data[(data['ORIGIN']==origin) & (data['DEST']==destination) & (data['MONTH']==month)]
#         numflights = subdata.shape[0]
#         numcarriers = len(subdata['UNIQUE_CARRIER_NAME'].unique())
#     else:
#         subdata = data[(data['ORIGIN']==origin) & (data['DEST']==destination) & (data['UNIQUE_CARRIER_NAME']==carrier) & (data['MONTH']==month)]
#         numflights = subdata.shape[0]
#         numcarriers = 1
#     return [subdata, numflights, numcarriers]

In [ ]:
# data = pd.read_csv('All_Flights.csv')
# [subdata, numflights, numcarriers] = getflightdata(data, 'LAX', 'JFK', 'No Selection', 1)

In [ ]:
# encoder = pickle.load(open('OneHotEncoder.pkl', 'rb'))
# rfr = pickle.load(open('Load_Factor_RFR_Model.pkl','rb'))
# X = encoder.transform(subdata)
# predictions = rfr.predict(X)

In [ ]:
# predictions

In [ ]:
# planetypes = pd.read_csv('L_AIRCRAFT_TYPE.csv')
# subdata = subdata.merge(planetypes,how='left',left_on='AIRCRAFT_TYPE',right_on='Code')
# subdata['Predictions']=predictions.astype(np.int64)
# finaldata = subdata[['UNIQUE_CARRIER_NAME','Description','Predictions']]


In [ ]:
# finaldata

In [68]:
data[(data['LOAD_FACTOR']<0.5)&(data['DEST']=='ANC')&(data['MONTH']==8)]

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,...,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,Unnamed: 45,LOAD_FACTOR
38122,0.0,1.0,3000.0,9.0,0.0,0.0,0.0,157.0,38.0,34.0,...,4,479,1,2019,3,8,1,L,NaN,0.000000
38123,0.0,1.0,3000.0,9.0,4.0,0.0,0.0,424.0,110.0,106.0,...,4,479,1,2019,3,8,1,L,NaN,0.444444
38125,0.0,5.0,15000.0,45.0,17.0,0.0,0.0,329.0,402.0,382.0,...,4,479,1,2019,3,8,1,L,NaN,0.377778
38127,0.0,1.0,3000.0,9.0,3.0,0.0,0.0,240.0,60.0,56.0,...,4,479,1,2019,3,8,1,F,NaN,0.333333
38138,0.0,2.0,6000.0,18.0,0.0,0.0,0.0,195.0,95.0,87.0,...,4,479,1,2019,3,8,1,L,NaN,0.000000
38184,0.0,1.0,3000.0,9.0,0.0,0.0,0.0,288.0,71.0,67.0,...,4,479,1,2019,3,8,1,P,NaN,0.000000
38231,0.0,2.0,6000.0,18.0,0.0,0.0,0.0,626.0,280.0,272.0,...,4,479,1,2019,3,8,2,P,NaN,0.000000
44684,0.0,4.0,22400.0,3.0,0.0,0.0,0.0,253.0,398.0,374.0,...,4,412,2,2019,3,8,1,G,NaN,0.000000
44685,0.0,17.0,127500.0,6.0,0.0,0.0,0.0,253.0,1218.0,1066.0,...,4,459,2,2019,3,8,1,G,NaN,0.000000
44921,0.0,1.0,7500.0,1.0,0.0,0.0,0.0,59.0,32.0,23.0,...,4,459,2,2019,3,8,1,G,NaN,0.000000


In [67]:
data.iloc[314903]

DEPARTURES_SCHEDULED                     35.0
DEPARTURES_PERFORMED                     43.0
PAYLOAD                             1989700.0
SEATS                                  7654.0
PASSENGERS                             3511.0
FREIGHT                                3297.0
MAIL                                   4302.0
DISTANCE                                261.0
RAMP_TO_RAMP                           2860.0
AIR_TIME                               1946.0
UNIQUE_CARRIER                             AS
AIRLINE_ID                              19930
UNIQUE_CARRIER_NAME      Alaska Airlines Inc.
UNIQUE_CARRIER_ENTITY                   06031
REGION                                      D
CARRIER                                    AS
CARRIER_NAME             Alaska Airlines Inc.
CARRIER_GROUP                               3
CARRIER_GROUP_NEW                           3
ORIGIN_AIRPORT_ID                       11630
ORIGIN_AIRPORT_SEQ_ID                 1163002
ORIGIN_CITY_MARKET_ID             